In [2]:
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import torch
import numpy as np
from torch import Tensor, nn

# Implementazione di regressore softmax
Carichiamo dataset delle iris di fisher. Contenente
- 4 quantità relative (features)
- 150 fiori (classi)
- 3 specie diverse (istanze)


In [3]:
iris = load_iris()
X=iris.data
Y=iris.target
print("features", X.shape,
    "\n classi target", Y.shape,
    "\n", Y)

features (150, 4) 
 classi target (150,) 
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [4]:
## Inizializzazione

## seed per risultati ripetibili
np.random.seed(1234)
torch.random.manual_seed(1234)

## permutazione casuale dei dati
idx = np.random.permutation(len(X))

## applico la stessa sia a X che a Y
X = X[idx]
Y = Y[idx]

## suddivido dataset in training e testing set indipendenti e trasformiamo gli array in tensori
X_training = Tensor(X[30:])
Y_training = Tensor(Y[30:])
X_testing = Tensor(X[:30])
Y_testing = Tensor(Y[:30])

## normalizzo i dati
X_mean = X_training.mean(0)
X_std = X_training.std(0)

X_training_norm = (X_training-X_mean)/X_std
X_testing_norm = (X_testing-X_mean)/X_std


Definisco un nuovo modulo per effettuare la regressione softmax 

In [5]:
class SoftMaxRegressor(nn.Module):
    def __init__(self, in_features, out_classes):
        """Costruisce un regressore softmax
            Input:
                in_features: numero di feature in input (es.4)
                out_classes: numero di classi in uscita (es.3) """
        super(SoftMaxRegressor, self).__init__()    ## richiamo costruttore della superclasse, passo necessario per abilitare alcuni meccanismi automatici di PyTorch

        self.linear = nn.Linear(in_features, out_classes)   ## il regressore softmax restituisce distr di probabilità, quindi il numero di feature di output coincide con il numero di classi. è lineare in quanto il softmax viene implementato nella loss

    def forward(self,x):
        """Definisce come processare l'input x"""
        scores = self.linear(x)
        return scores

Costruiamo un regressore softmax e passiamogli i dati di training

In [6]:
## z = torch.Tensor([14,4.3, 100])

## implementazione grezza del softmax
## def softmax(z):
##    z = z-torch.max(z)      ## permette che sia più robusta per i numeri più grandi
##    z_exp = torch.exp(z)
##    return z_exp/z_exp.sum()

## print(softmax(z))

model = SoftMaxRegressor(4,3) # 4 feature in ingresso, 3 classi in uscita
#mostriamo le prime 4 predizioni
model(X_training_norm)[:10]

tensor([[ 0.9326,  0.6582, -0.2940],
        [-0.1301, -0.2885,  0.1746],
        [ 1.1440,  1.6170, -1.0236],
        [-0.1766, -0.4061,  0.2196],
        [-0.4706, -0.5166,  0.2307],
        [ 1.2162,  1.5326, -0.9408],
        [ 1.6219,  1.6549, -0.9210],
        [ 0.8984,  1.2598, -0.8457],
        [ 1.4105,  1.9569, -1.2018],
        [ 1.0097,  0.5520, -0.2012]], grad_fn=<SliceBackward>)

ogni riga della matrice è una predizione. Non si tratta di valide distribuzioni di probabilità, per ottenere le distribuzioni usiamo softmax

In [7]:
softmax = nn.Softmax(dim=1)
#softmax(model(X_training_norm))[:10]

## adesso abbiamo una valida distribuzioni di probabilità sulle tre classi. la somma lunghe le righe è pari a 1 infatti:

#softmax(model(X_training_norm)).sum(1)

una volta allenato, il modello permetterà di predire una distribuzione di probabilità per ogni elemento. per ottenere l'etichetta predetta, applichiamo il principio Maximum A Posteriori (MAP), scegliendo la classe che presenta la probabilità maggiore mediante argmax inclusa in pytorch nella funzione max

In [8]:
# ritorna i valori dei massimi e i loro indici (il ris della funzione argmax)
# per questo includiamo [1] nell'equazione successiva

preds = softmax(model(X_training_norm)).max(1)[1]
preds

## dopo aver ottenuto le predizioni sotto forma di indici delle rte classi che vanno da 0 a 2 possiamo valutare le predizioni come visto nel caso binario. calcoliamo l'accuracy

print(accuracy_score(Y_training, preds))

0.35833333333333334


L'accuracy è molto bassa in quanto dobbiamo ancora allenare il modello

Dato che la funzione softmax è monotona, possiamo applicare argmax direttamente ai logits ottenendo lo stesso risultato

```
preds_logits= model(X_training_norm).max(1)[1]
print((preds_logits==preds).float().mean()) #il risultato ottenuto è lo stesso
```

In pratica si preferisce non applicare softmax per il calcolo delle etichette predette
(mancano le formulette)
La procedura di training del regressore logistico sarà la seguente:
    1.Normalizzare i dati in ingresso  
    2. Costruire il modulo che implementa il modello (il costruttore si preoccuperà di inizializzare i parametri)
    3. Mettere il modello in modalità "training"
    4. Calcolare l'output del modello  
    5. Calcolare il valore della loss 
    6. Calcolare il gradiente della loss rispetto ai parametri del modello;
    7. Aggiornare i pesi   utilizzando il gradient descent
    8. Ripetere i passi 4-7 fino a convergenza.

Implementiamo di conseguenza la procedura introducendo il monitoring delle curve tramite tensorboard e calcolo dell'accuracy ad ogni iterazione

In [9]:
from torch.utils.tensorboard import SummaryWriter
from torch.optim import SGD

writer = SummaryWriter('logs/softmax_regressor')

lr = 0.1
epochs = 500

## normalizzazione dei dati
X_mean = X_training.mean(0)
X_std = X_training.std(0)

X_training_norm = (X_training-X_mean)/X_std
X_testing_norm = (X_testing-X_mean)/X_std

model = SoftMaxRegressor(4, 3)
criterion = nn.CrossEntropyLoss()       # cross-entropy loss
optimizer = SGD(model.parameters(), lr)  # optimizer

for e in range(epochs):
    model.train()
    out = model(X_training_norm)
    l = criterion(out, Y_training.long())
    l.backward()
    writer.add_scalar('loss/train', l.item(), global_step=e)

    optimizer.step()
    optimizer.zero_grad()

    preds_train = out.max(1)[1]
    writer.add_scalar('accuracy/train', accuracy_score(Y_training, preds_train), global_step=e)

    model.eval()

    with torch.set_grad_enabled(False):
        out = model(X_testing_norm)
        l = criterion(out, Y_testing.long())
        writer.add_scalar('loss/test', l.item(), global_step=e)
        preds_test = out.max(1)[1]
        writer.add_scalar('accuracy/test', accuracy_score(Y_testing, preds_test), global_step=e)


## Calcolo accuracy di training e test

preds_train = model(X_training_norm).max(1)[1]
preds_test = model(X_testing_norm).max(1)[1]

print("Accuracy di training", accuracy_score(Y_training,preds_train) )
print("Accuracy di test", accuracy_score(Y_testing,preds_test) )


INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
Accuracy di training 0.9583333333333334
Accuracy di test 1.0


## Salvataggio e caricamento di modelli

Quando si allenano modelli su grandi dataset, la procedura di allenamento può essere molto lenta. Risulta dunqueconveniente poter salvare su disco i modelli in modo da poterli caricare e riutilizzare in seguito. PyTorch permette di salvare ecaricare modelli in maniera semplice. Il salvataggio viene effettuato serializzando tutti i parametri. E' possibile accedere a undizionario contenente tutti i parametri del modello utilizzando il metodo state_dict

In [10]:
state_dict=model.state_dict()
print(state_dict.keys())

odict_keys(['linear.weight', 'linear.bias'])


Nel nostro caso si tratta di due soli elementi ma in generale potrebbero essercene di più. Possiamo dunque salvare il dizionario tramite torch.save:


In [11]:
torch.save(model.state_dict(),'model.pth')

per ripristinare lo stato del modello, dobbiamo prima costruire l'oggetto e poi usare il metodo load_state_dict

In [12]:
model = SoftMaxRegressor(4,3)
model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

## Allenamento su GPU

Dato che l'allenamento di un modello su grandi quantità di dati può essere lento, risulta conveniente velocizzare i calcoli effettuando l'allenamento su GPU, qualora una GPU dovesse essere disponibile nel sistema. Vediamo alcuni semplici passi per convertire il codice di training in questo senso

E' possibile verificare qualora una GPU sia disponibile nel sistema:


In [13]:
torch.cuda.is_available()

False

Possiamo dunque costruire una variabile device che sia uguale a cpu se non c'è nessuna GPU disponibile e cuda altrimenti

In [15]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [16]:
print(device)

cpu


Dobbiamo "portare" il modello che utilizzeremo sul device corretto:

In [ ]:
mode.to(device)

La stessa operazione va effettuata su ciascun tensore con il quale lavoreremo come segue:

In [18]:
X_training_norm.to(device)

tensor([[-0.8943, -1.2295, -0.4128, -0.1262],
        [ 0.4487, -0.5708,  0.6062,  0.7799],
        [-1.0163,  0.9660, -1.3753, -1.1617],
        [ 0.5707, -0.5708,  0.7761,  0.3915],
        [ 1.0591, -0.1317,  0.7195,  0.6504],
        [-1.2605,  0.7465, -1.0356, -1.2911],
        [-1.7488, -0.3513, -1.3187, -1.2911],
        [-0.5280,  0.7465, -1.1488, -1.2911],
        [-1.5047,  1.1856, -1.5451, -1.2911],
        [-1.0163, -1.6686, -0.2430, -0.2556],
        [-0.4059,  0.9660, -1.3753, -1.2911],
        [ 0.4487, -1.8881,  0.4364,  0.3915],
        [-0.7722,  2.2833, -1.2620, -1.4206],
        [ 1.3032,  0.0878,  0.7761,  1.4270],
        [ 1.6695,  0.3074,  1.2856,  0.7799],
        [-1.1384,  0.0878, -1.2620, -1.4206],
        [-0.0397, -0.5708,  0.7761,  1.5565],
        [ 1.0591, -0.1317,  0.8327,  1.4270],
        [ 2.2799, -0.1317,  1.3422,  1.4270],
        [ 1.0591,  0.0878,  1.0592,  1.5565],
        [-1.1384, -0.1317, -1.3187, -1.2911],
        [ 0.8149, -0.1317,  0.8327